<a href="https://colab.research.google.com/github/puraminy/mini_proj1/blob/master/POS_TAGGER_RNN_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')
data_path = 'gdrive/My Drive/NLP-CA4'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
# -*- coding: utf-8 -*-
"""NLP_CA4_RNN .ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/181q3MmuRBQgMva2FXWVHoM0l0aqiiPwr
"""

#from google.colab import drive
#drive.mount('/content/gdrive')


#data_path = '.'

# -*- coding: utf-8 -*-
"""NLP_CA4_2 .ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/181q3MmuRBQgMva2FXWVHoM0l0aqiiPwr
"""
# from google.colab import drive
# drive.mount('/content/gdrive')
import numpy as np
#data_path = '.'
def to_categorical(sequences, categories):
    cat_sequences = []
    for s in sequences:
        cats = []
        for item in s:
            cats.append(np.zeros(categories))
            cats[-1][item] = 1.0
        cat_sequences.append(cats)
    return np.array(cat_sequences)

def ignore_class_accuracy(to_ignore=0):
    def ignore_accuracy(y_true, y_pred):
        y_true_class = K.argmax(y_true, axis=-1)
        y_pred_class = K.argmax(y_pred, axis=-1)

        ignore_mask = K.cast(K.not_equal(y_pred_class, to_ignore), 'int32')
        matches = K.cast(K.equal(y_true_class, y_pred_class), 'int32') * ignore_mask
        accuracy = K.sum(matches) / K.maximum(K.sum(ignore_mask), 1)
        return accuracy
    return ignore_accuracy
        
def extract_sentences(wordList):
  nonBreak_delims = ['،','؛',':']
  sentences = []
  sentences_tags = []
  sent = []
  sent_tag = []  
  for line in wordList:
    tag = line[-1]
    word = ''.join(line[:-1])
    if tag == "DELM" and word not in nonBreak_delims:
      if len(sent) > 0:
        sentences.append(sent)
        sentences_tags.append(sent_tag)
      sent = []
      sent_tag = []
    else:
      sent.append(word)
      sent_tag.append(tag)
  return sentences, sentences_tags


from sklearn.model_selection import StratifiedKFold
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

def prepare_data(train_sentences, train_tags):
      words, tags = set([]), set([])
      for s in train_sentences:
          for w in s:
              words.add(w.lower())

      for ts in train_tags:
          for t in ts:
              tags.add(t)

      word2index = {w: i + 2 for i, w in enumerate(list(words))}
      word2index['-PAD-'] = 0  # The special value used for padding
      word2index['-OOV-'] = 1  # The special value used for OOVs

      tag2index = {t: i + 1 for i, t in enumerate(list(tags))}
      tag2index['-PAD-'] = 0  # The special value used to padding
      tag2index['-OOT-'] = -1 

      sentences_X, tags_Y= [], []

      for s in train_sentences:
          s_int = []
          for w in s:
              try:
                  s_int.append(word2index[w.lower()])
              except KeyError:
                  s_int.append(word2index['-OOV-'])

          sentences_X.append(s_int)

      for s in train_tags:
          tags_y = []
          for t in s:
              try:
                 tags_y.append(tag2index[t])
              except KeyError:
                  tags_y.append(tag2index['-OOT-'])
          tags_Y.append(tags_y)

      MAX_LENGTH = len(max(sentences_X, key=len))
      sentences_X = pad_sequences(sentences_X, maxlen=MAX_LENGTH, padding='post')
      tags_Y = pad_sequences(tags_Y, maxlen=MAX_LENGTH, padding='post')
      
      return sentences_X, tags_Y, word2index, tag2index

from keras.models import Sequential
from keras.layers import Dense, LSTM, InputLayer, Bidirectional, TimeDistributed, Embedding, Activation
from keras.optimizers import Adam
from keras import backend as K
from keras.models import load_model
import matplotlib.pyplot as plt
import pickle

# ============================================= Training and Testing ================================
use_saved_model = False
train_from = 1
train_to = 500000 # 
model_id = str(train_from)+ "-" + str(train_to)

if not use_saved_model:
    trainFileName = data_path + '/train.txt'
    trainFile = open(trainFileName,'r', encoding="utf-8")
    testFile = open(data_path + '/test.txt','w', encoding="utf-8")
    wordList = []
    EPOCHS = 6
    test_from = 1
    test_to = 1000
    i = 0
    for lines in trainFile:
        i+=1
        if i in range(test_from, test_to):
            testFile.writelines(lines)
        elif i in range(train_from, train_to):
            wordList.append(lines.split())
        else:
          break
    trainFile.close()
    testFile.close()

    sentences, sentence_tags =extract_sentences(wordList)
    print("Sample sentencs")
    for k in range(2):
      print(sentences[k])
      print(sentence_tags[k])

    sentences_X, tags_Y, word2index, tag2index = prepare_data(sentences, sentence_tags)
    MAX_LENGTH = len(max(sentences_X, key=len))
    # (train_sentences, test_sentences, train_tags, test_tags) = train_test_split(sentences, sentence_tags, test_size=0.2)    
    kFold = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)
    print("X shape:", sentences_X.shape)
    print("Y shape:", tags_Y.shape)
    cat_train_y = to_categorical(tags_Y, len(tag2index))
    print("Y shape:", cat_train_y.shape)
    
    # Saving word indexes to load them later without creating them again
    wordIndexFilename = data_path + "/word2index-" + model_id + ".txt"
    print("Saving word indexs ", wordIndexFilename)
    tagIndexlFilename = data_path + "/tag2index-" + model_id + ".txt"
    print("Saving word indexs ", tagIndexlFilename)
    with open(wordIndexFilename, "wb") as fp:   #Pickling
        pickle.dump(word2index, fp)
    with open(tagIndexlFilename, "wb") as fp:   #Pickling
        pickle.dump(tag2index, fp) 
    hist = []
    fold = 0
    for train, test in kFold.split(sentences_X, tags_Y.argmax(1)):
        fold += 1
        print("Fold:", fold)
        print("Preparing data ....")
        # ================================================== MODEL ============================
        MAX_LENGTH = len(max(sentences_X[train], key=len))
        print("Training and testing ....")

        model = Sequential()
        model.add(InputLayer(input_shape=(MAX_LENGTH, )))
        model.add(Embedding(len(word2index), 128))
        model.add(Bidirectional(LSTM(256, return_sequences=True)))
        model.add(TimeDistributed(Dense(len(tag2index))))
        model.add(Activation('softmax'))

        model.compile(loss='categorical_crossentropy',
                      optimizer=Adam(0.001),
                      metrics=['accuracy', ignore_class_accuracy(0)])

        model.summary()
        cat_tags_y = to_categorical(tags_Y, len(tag2index))
        model.fit(sentences_X[train], cat_tags_y[train],  
                  batch_size=128, epochs=EPOCHS)
        scores = model.evaluate(sentences_X[test], cat_tags_y[test])
        print(f"Validation score: {model.metrics_names}:", scores)
        savedModelFilename = data_path + "/model-" + model_id+ "-fold" + str(fold) + ".h5"
        print("Saving model in ", savedModelFilename)
        model.save(savedModelFilename)
        hist.append(scores)
        with open(data_path + "/hist-"+model_id+ ".txt", "wb") as fp:   #pickling
            pickle.dump(hist, fp)  
        
    print(hist)
    savedModelFilename = data_path + "/model-"+model_id+".h5"
    print("Saving model in ", savedModelFilename)
    model.save(savedModelFilename)
else:
  with open(data_path + "/word2index-" + model_id + ".txt", "rb") as fp:   # Unpickling
      word2index = pickle.load(fp)  
  with open(data_path + "/tag2index-" + model_id + ".txt", "rb") as fp:   # Unpickling
      word2index = pickle.load(fp)  
  model = load_model(data_path + "/model-"+ model_id + "-fold1.h5", {"ignore_accuracy":ignore_class_accuracy()})
      


Sample sentencs
['به', 'تصوير', 'كند', 'و', 'ظاهرا', 'هيچگاه', 'مانع', 'و', 'محدوديتي', 'هم', 'براي', 'ارااه', 'اين', 'ايدهها', 'وجود', 'نداشته', 'است']
['P', 'N_SING', 'V_SUB', 'CON', 'ADV_NI', 'ADV_TIME', 'N_SING', 'CON', 'N_SING', 'CON', 'P', 'N_SING', 'DET', 'N_PL', 'N_SING', 'ADJ_INO', 'V_PRE']
['بورن', 'هرگاه', 'احساس', 'كند', 'دنياي', 'كاريكاتور', 'جوابگوي', 'او', 'نيست', '،', 'به', 'نقاشي', 'پناه', 'ميبرد', 'و', 'هنگاميكه', 'ميخواهد', 'موجوداتش', 'را', 'به', 'گونهاي', 'زندهتر', 'حس', 'كند', '،', 'به', 'عنوان', 'انيماتور', 'به', 'كار', 'مشغول', 'ميشود']
['N_SING', 'ADV_TIME', 'N_SING', 'V_SUB', 'N_SING', 'N_SING', 'N_SING', 'PRO', 'V_PRS', 'DELM', 'P', 'N_SING', 'N_SING', 'V_PRS', 'CON', 'CON', 'V_PRS', 'N_PL', 'P', 'P', 'N_SING', 'ADJ_CMPR', 'N_SING', 'V_SUB', 'DELM', 'P', 'N_SING', 'N_SING', 'P', 'N_SING', 'ADJ_SIM', 'V_PRS']
X shape: (24594, 253)
Y shape: (24594, 253)
Y shape: (24594, 253, 40)
Saving word indexs  gdrive/My Drive/NLP-CA4/word2index-1-500000.txt
Saving word ind

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 253, 128)          3951232   
_________________________________________________________________
bidirectional_8 (Bidirection (None, 253, 512)          788480    
_________________________________________________________________
time_distributed_8 (TimeDist (None, 253, 40)           20520     
_________________________________________________________________
activation_8 (Activation)    (None, 253, 40)           0         
Total params: 4,760,232
Trainable params: 4,760,232
Non-trainable params: 0
_________________________________________________________________
Epoch 1/6
19665/19665 [==============================] - 870s 44ms/step - loss: 0.3337 - acc: 0.9419 - ignore_accuracy: 0.3384
Epoch 2/6
 7808/19665 [==========>...................] - ETA: 8:41 - loss: 0.1638 - acc: 0.9534 - ignore_accuracy: 0.3931

In [0]:

#=================================================== Testing the model

def logits_to_tokens(sequences, index):
    token_sequences = []
    for categorical_sequence in sequences:
        token_sequence = []
        for categorical in categorical_sequence:
            m = np.argmax(categorical)
            token_sequence.append(index[m])
        token_sequences.append(token_sequence) 
    return token_sequences
  
# This is a sample file for testing the model
testFileName = data_path + '/test.txt'
testFile = open(testFileName, 'r', encoding="utf-8")
wordList = []
for lines in testFile:
    wordList.append(lines.split())
testFile.close()


test_samples, test_tags =extract_sentences(wordList)

test_samples_X = []
for s in test_samples:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
    test_samples_X.append(s_int)


test_samples_X = pad_sequences(test_samples_X, maxlen=MAX_LENGTH, padding='post')

predictions = model.predict(test_samples_X)
tagDict = {i: t for t, i in tag2index.items()}

y_pred = logits_to_tokens(predictions, tagDict)

outFilename = data_path + '/results.txt'
outFile = open(outFilename, 'w', encoding="utf-8")

y_pred_list = []
y_test_list = []
k = 1
for i in range(len(test_samples)):
  yp = y_pred[i]
  ya = test_tags[i]
  for j,x in enumerate(ya):
    y_test_list.append(x)
    y_pred_list.append(yp[j])
    outFile.writelines(f"{k}\t{yp[j]}\t{x}\n")
    k+=1

    
outFile.close()
print("Number of Tags:", len(y_test_list))

print("Actual:",y_test_list)
print("Predicted:",y_pred_list)
from sklearn.metrics import classification_report


print(classification_report(y_test_list, y_pred_list))



In [0]:
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels

def unique(list1):      
    # insert the list to the set 
    list_set = set(list1) 
    # convert the set to the list 
    unique_list = (list(list_set)) 
    return unique_list
  
y_test = y_test_list
y_pred = y_pred_list
class_names = sorted(unique(y_test_list))

# Plot non-normalized confusion matrix
# plot_confusion_matrix(y_test_list, y_pred_list, classes=y_test_list,
#                       title='Confusion matrix, with normalization')
import seaborn as sn
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score
cm = confusion_matrix(y_test, y_pred, class_names)
normalize = True
format = "d"
if normalize:
    cm = 100*(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis])
    format = '0.0f'
    
print("Confusion Matrix:")

aylabels = class_names
axlabels = class_names

plt.figure(figsize=(20, 15))
plt.title("Confusion Matrix")
sn.set(font_scale=1.4)  # for label size
sn.heatmap(cm, annot=True,fmt=format, annot_kws={"size": 12}, xticklabels=axlabels, yticklabels=aylabels)  # font size
plt.show()